# <h1><center>CodeIT AI Engineer 3기 Mission 4</center></h1>



### 문제
여러분은 지금부터 포르투갈 은행의 마케팅 담당자입니다.
<br />

데이터는 2008년부터 2010년까지의 은행 마케팅 캠페인 데이터를 포함하고 있습니다. 
<br />

여러분의 목표는 이 데이터를 통해 고객이 정기 예금을 가입할 가능성을 예측하고, 이를 통해 마케팅 캠페인의 효율성을 높이는 것입니다. 
<br />

마케팅 담당자로서 정기 예금과 관련이 있는 요소들을 파악해보고, 고객의 행동을 이해해보세요. 어떤 상황에서 어떤 고객들이 정기 예금을 가입할까요?
<br /><br />

이번 미션의 최종 목표는 가장 정확한 분류 모델을 개발하여 고객이 정기 예금을 가입할지 여부를 예측하고, 그 모델을 통해 도출한 인사이트를 바탕으로 비즈니스 전략을 제시하는 것입니다.

<br />

---




### 목표

결정 트리와 앙상블 기법을 사용해 가장 정확한 분류 모델을 구축하고, 고객이 정기 예금을 가입할지 여부를 예측해 마케팅 캠페인의 효율성을 높이는 인사이트를 도출해 비즈니스 전략을 제시하세요.

<br />

---



### 데이터 SET
<br />

|컬럼명|설명|
|---|---|
|age|	나이 (숫자)|
|job|	직업 (범주형)|
|marital|	결혼 여부 (범주형)|
|education|	교육 수준 (범주형)|
|default|	신용 불량 여부 (범주형)|
|housing|	주택 대출 여부 (범주형)|
|loan|	개인 대출 여부 (범주형)|
|contact|	연락 유형 (범주형)|
|month|	마지막 연락 월 (범주형)|
|day_of_week|	마지막 연락 요일 (범주형)|
|duration|	마지막 연락 지속 시간, 초 단위 (숫자)|
|campaign|	캠페인 동안 연락 횟수 (숫자)|
|pdays|	이전 캠페인 후 지난 일수 (숫자)|
|previous|	이전 캠페인 동안 연락 횟수 (숫자)|
|poutcome|	이전 캠페인의 결과 (범주형)|
|emp.var.rate|	고용 변동률 (숫자)|
|cons.price.idx|	소비자 물가지수 (숫자)|
|cons.conf.idx|	소비자 신뢰지수 (숫자)|
|euribor3m|	3개월 유리보 금리 (숫자)|
|nr.employed|	고용자 수 (숫자)|
|y|	정기 예금 가입 여부 (이진: yes=1, no=0)|

해당 데이터는 UC Irvine Machine Learning Repository에서 제공하는 Bank Marketing 데이터 입니다. 

여러 데이터 중 2014년 아래 논문에서 사용된 데이터를 선택하였습니다. 

데이터에 대해서 더 자세히 알고 싶은 분들은 아래 데이터 설명 txt파일과 논문을 확인해보시는 걸 추천드립니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from matplotlib import rc  ### 이 줄과
rc('font', family='AppleGothic') 			## 이 두 줄을 
plt.rcParams['axes.unicode_minus'] = False  ## 추가해줍니다. 
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# read dataset
train_df = pd.read_csv("./bank-additional-full.csv", sep=';')

## 1. 데이터 탐색

### 데이터 확인

In [ ]:
print(train_df.shape)
train_df.head(5)

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
# 결측값 확인
print(train_df.isnull().sum())

In [ ]:
# 중복값 확인
print(train_df.duplicated().sum())

### 데이터 1차 확인 결과

- train: 전체 41188행 21열의 구조
- 범주형 변수가 11개, 수치형 변수가 10개로 확인됨
- 타겟 변수: y (정기 예금 가입 여부 - yes/no)
- 주요 특성: 나이, 직업, 결혼 여부, 신용 불량 여부, 마지막 연락 지속 시간, 캠페인 동안 연락 횟수, 고용 변동률, 소비자 물가지수, 소비자 신뢰지수, 3개월 유리보 금리, 고용자 수
---

- 중복값 12 행이 확인됨
- 결측값은 발견되지 않음

- 확인할 컬럼들:
	- duration 열은 평균과 min, max 값이 큰 차이를 보임 (마지막 연락 지속 시간)
	- campaign 열은 평균과 min, max 값이 큰 차이를 보임 (캠페인 동안 연락 횟수)
	- pdays 열은 999값이 아주 많은것으로 확인됨 
	- previous 열은 0값이 많은것으로 확인됨 (pdays와 관련이 있을 것으로 추정됨)

- 종속 변수 y열은 데이터 확인 후 이진화가 필요해보임

- 범주형 변수들은 모델링을 위해 인코딩으로 변환 필요

## 2. 데이터 전처리

### 범주형 열 시각화

In [ ]:
# month 열의 카테고리 순서 지정
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# 범주형 열의 카운트 플롯 그리기
cat_cols = train_df.select_dtypes(include=['object']).columns
n_cols = 3
n_rows = int(np.ceil(len(cat_cols) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 3 * n_rows))

for idx, col in enumerate(cat_cols):
	row = idx // n_cols
	col_idx = idx % n_cols
	ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
	if col == 'month':
		sns.countplot(data=train_df, x=col, hue='y', ax=ax, order=month_order)
	else:
		sns.countplot(data=train_df, x=col, hue='y', ax=ax)
	ax.set_title(f'Count Plot of {col}')
	ax.set_xlabel(col)
	ax.set_ylabel('Count')
	ax.tick_params(axis='x', rotation=45)
# 빈 subplot 숨기기
for idx in range(len(cat_cols), n_rows * n_cols):
	row = idx // n_cols
	col_idx = idx % n_cols
	ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
	ax.axis('off')

plt.tight_layout()
plt.show()

- 범주형 열의 데이터에 'unknown' 값이 존재함을 확인
- 직업별 정기 예금 가입률은 유의미해 보이지 않음을 확인
- 결혼의 유무도 유의미하게 차이나 보이지는 않음
- 교육 수준별 정기 예금 가입률도 유의미해 보이지는 않음, 'illiterate'는 극소수만이 존재함을 확인
- 신용불량여부 열
	- 'unknown' 값의 비율이 높지만, 상식적으로 생각해보았을때, 신용불량자가 정기예금을 가입할 가능성은 낮을 것으로 판단됨
	- 모델링시 열 자체를 제거하는 것이 좋을 것으로 판단됨
- 주택대출 여부는 균등하게 분포되어 있음
- 개인대출을 받지않은 상담자가 많지만, 정기 예금 가입률은 유의미한 차이를 보이지 않음
- 연락 유형은 'cellular'가 많고, 정기 예금 가입률도 높음
- 월별 연락 수는 1, 2월에는 정보가 없음을 확인, 연락 수가 고루 분포되어 있지 않음
- poutcome 열을 확인 했을때, 'nonexistent'가 많은 것으로 보아 이번 캠페인이 첫 상담인 경우가 많음을 확인

### 수치형 열 시각화

In [ ]:
# 숫자형 열의 히스토그램 (y에 따라 분리, count 기준)
num_cols = train_df.select_dtypes(include=['int64', 'float64']).columns

n_cols = 3
n_rows = int(np.ceil(len(num_cols) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 2 * n_rows))

for idx, col in enumerate(num_cols):
	row = idx // n_cols
	col_idx = idx % n_cols
	ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
	# y별로 count 기준 히스토그램
	sns.histplot(data=train_df, x=col, hue='y', bins=30, ax=ax, element='step', stat='count', multiple='dodge')
	ax.set_title(f'Histogram of {col} by y')
	ax.set_xlabel(col)
	ax.set_ylabel('Count')
	# ax.set_yscale('log')  # y축을 로그 스케일로 설정

# 빈 subplot 숨기기
for idx in range(len(num_cols), n_rows * n_cols):
	row = idx // n_cols
	col_idx = idx % n_cols
	ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
	ax.axis('off')

plt.tight_layout()
plt.show()


- age
	- 20대 중반부터 50대 후반까지가 주를 이루고 있음
	- 정기 예금 가입률은 30대 후반에서 40대 초반이 가장 높음
- duration, campaign열을 보니, 2~3번 연락 중 2~5분의 통화 사이에 정기 예금 가입률이 가장 높음
- pdays, previous 열을 보니, pdays가 999인 경우가 많고, previous가 0인 경우가 많음
	- pdays가 999인 경우는 이전 캠페인에서 연락이 없었던 경우로 판단됨
	- previous가 0인 경우는 이번 캠페인이 첫 상담인 경우로 판단됨
- 경제 지표인 emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed 열을 확인했을때 뚜렷한 인사이트는 발견되지 않음


### 이상치 시각화

In [ ]:
# 이상치 시각화 (한 줄에 3개씩)
def plot_boxplot_grid(df, num_cols, n_cols=3):
	n_rows = int(np.ceil(len(num_cols) / n_cols))
	fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 1.5 * n_rows))
	for idx, col in enumerate(num_cols):
		row = idx // n_cols
		col_idx = idx % n_cols
		ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
		sns.boxplot(x=df[col], ax=ax)
		ax.set_title(f'Boxplot of {col}')
		ax.set_xlabel(col)
	# 빈 subplot 숨기기
	for idx in range(len(num_cols), n_rows * n_cols):
		row = idx // n_cols
		col_idx = idx % n_cols
		ax = axes[row, col_idx] if n_rows > 1 else axes[col_idx]
		ax.axis('off')
	plt.tight_layout()
	plt.show()

plot_boxplot_grid(train_df, num_cols)

In [ ]:
# 범주형 열의 'unknown' 값 시각화
def plot_unknown_counts(df, cat_cols):
	unknown_counts = {col: df[col].value_counts().get('unknown', 0) for col in cat_cols}
	# 0이 아닌 값만 필터링
	unknown_counts = {col: count for col, count in unknown_counts.items() if count > 0}
	unknown_df = pd.DataFrame(list(unknown_counts.items()), columns=['Column', 'Unknown Count'])

	plt.figure(figsize=(5, 3))
	sns.barplot(data=unknown_df, x='Column', y='Unknown Count')
	plt.xticks(rotation=45)
	plt.title('Count of Unknown Values in Categorical Columns')
	plt.xlabel('Column')
	plt.ylabel('Unknown Count')
	plt.tight_layout()
	plt.show()

# 범주형 열의 'unknown' 값 시각화
plot_unknown_counts(train_df, cat_cols)

### 이상치 처리

- age 열은 정기 예금 상품 특성상 나이가 너무 낮거나 많을수록 예금 상품에 가입할 확률이 낮다고 판단, iqr 방식으로 제거
- 상담 시간은 2~5분 사이에 정기 예금 가입 의사결정이 내려지는 것으로 보임. 이상치 기준을 러프하게 정의하여 10분 이상인 경우 제거
- 상담 횟수가 너무 많으면 예금 상품에 가입할 확률이 낮다고 판단, iqr 방식으로 제거
- pdays, previous 열은 이번 캠페인의 신규 상담인경우가 많아 이상치 제거는 하지 않음

In [ ]:
# age 열 iqr 계산 및 제거 -- 정기 예금 상품 특성상 나이가 너무 낮거나 많을수록 예금 상품에 가입할 확률이 낮다고 판단
def remove_age_outliers(df, col):
	q1 = df[col].quantile(0.25)
	q3 = df[col].quantile(0.75)
	iqr = q3 - q1
	lower_bound = q1 - 1.5 * iqr
	upper_bound = q3 + 1.5 * iqr
	return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

train_df = remove_age_outliers(train_df, 'age')

# campaign 열 iqr 계산 및 제거 -- 연락을 많이할수록 예금 상품 가입률이 떨어진다고 판단
train_df = remove_age_outliers(train_df, 'campaign')

In [ ]:
# 'unknown' 값을 가진 행이 모두 몇 개인지 확인
def count_unknown_rows(df, cat_cols):
	unknown_rows = df[df[cat_cols].apply(lambda x: (x == 'unknown').any(), axis=1)]
	return unknown_rows.shape[0]

# 'unknown' 값을 가진 행의 개수 출력
unknown_count = count_unknown_rows(train_df, cat_cols)
print(f"'unknown' 값을 가진 행 수: {unknown_count}")

In [ ]:
# 상관관계 히트맵 그리기
corr_matrix = train_df.select_dtypes(include=['int64', 'float64']).corr()

plt.figure(figsize=(6, 5))
sns.set_theme(font_scale=0.8)
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Heatmap', fontsize=14)
plt.show()

In [ ]:
def plot_subscription_by_duration_with_rate(df):
	# duration을 100단위로 그룹화
	new_df = df.copy()  # 원본 데이터프레임을 변경하지 않도록 복사
	new_df['duration_bin'] = (new_df['duration'] // 100) * 100
	# 그룹별 가입여부(y) count 집계
	duration_counts = new_df.groupby(['duration_bin', 'y']).size().unstack(fill_value=0)

	# 가입률(yes 비율) 계산
	duration_counts['subscription_rate'] = duration_counts['yes'] / (duration_counts['yes'] + duration_counts['no'])

	# 시각화
	fig, ax1 = plt.subplots(figsize=(8, 4))

	# count bar
	duration_counts[['no', 'yes']].plot(
		kind='bar', 
		stacked=False, 
		color=[sns.color_palette('Set2')[1], sns.color_palette('Set2')[0]], 
		ax=ax1
	)
	ax1.set_xlabel('Duration (seconds, grouped by 100)')
	ax1.set_ylabel('Count')
	ax1.set_title('Subscription Count and Rate by Duration (100s)')
	ax1.legend(['No', 'Yes'], title='Subscribed')
	ax1.tick_params(axis='x', rotation=45)

	# 가입률(비율) 선그래프
	ax2 = ax1.twinx()
	ax2.plot(duration_counts.index.astype(str), duration_counts['subscription_rate'], 
			 color=sns.color_palette('Set2')[2], marker='o', label='Subscription Rate')
	ax2.set_ylabel('Subscription Rate')
	ax2.set_ylim(0, 1)
	ax2.legend(loc='upper right')

	plt.tight_layout()
	plt.show()

plot_subscription_by_duration_with_rate(train_df)

print('duration > 1300 의 y count: \n', train_df[train_df['duration'] > 1300]['y'].value_counts())

# duration > 1300인 행을 이상치로 판단해 제거
train_df.drop(train_df[train_df['duration'] > 1300].index, inplace=True)


In [ ]:
# 직업별/월별/기타 범주형 변수별 정기 예금 가입 시각화
def plot_job_subscription_count_and_rate(df, col):
	col_count = df.groupby(col)['y'].value_counts().unstack(fill_value=0)
	col_count['total'] = col_count['yes'] + col_count['no']
	# month면 month_order 순서로 정렬, 아니면 total 기준 내림차순
	if col == 'month':
		col_count = col_count.reindex(month_order)
	elif col == 'age':
		col_count = col_count.sort_values('age')
		
	col_count['subscription_rate'] = col_count['yes'] / col_count['total']

	fig, ax1 = plt.subplots(figsize=(8, 4))

	# 가입/비가입 count bar
	col_count[['no', 'yes']].plot(
		kind='bar',
		stacked=False,
		color=[sns.color_palette('Set2')[1], sns.color_palette('Set2')[0]],
		ax=ax1
	)
	ax1.set_ylabel('Count')
	ax1.set_title(f'Subscription Count and Rate by {col.capitalize()}')
	ax1.legend(['No', 'Yes'], title='Subscribed')
	ax1.tick_params(axis='x', rotation=45)

	# 가입률 선그래프
	ax2 = ax1.twinx()

	if col == 'age':
		ax2.errorbar(
			np.arange(len(col_count.index)), col_count["subscription_rate"],
			fmt='o', color=sns.color_palette('Set2')[2], markersize=5, label='Subscription Rate', linewidth=1.5, capsize=3
		)
	else:
		ax2.errorbar(
			col_count.index, col_count['subscription_rate'],
			fmt='o', color=sns.color_palette('Set2')[2], markersize=5, label='Subscription Rate', linewidth=1.5, capsize=3
		)
	ax2.set_ylabel('Subscription Rate')
	ax2.set_ylim(0, 1)
	ax2.legend()

	plt.tight_layout()
	plt.show()

plot_job_subscription_count_and_rate(train_df, 'job')

In [ ]:
# 교육 수준에 따라 정기 예금 가입률
plot_job_subscription_count_and_rate(train_df, 'education')

In [ ]:
# 월별 정기 예금 가입률의 차이
plot_job_subscription_count_and_rate(train_df, 'month')

In [ ]:
# 나이별 정기 예금 가입률의 차이
plot_job_subscription_count_and_rate(train_df, 'age')

### 데이터 타입 변환

In [ ]:
# y 열을 0과 1로 매핑
def map_y(y):
	return 1 if y == 'yes' else 0

# education 열은 교육 수준을 나타내며, 이를 숫자로 매핑합니다.
def map_education(education):
	if education == 'basic.4y':
		return 1
	elif education == 'basic.6y':
		return 2
	elif education == 'basic.9y':
		return 3
	elif education == 'high.school':
		return 4
	elif education == 'professional.course':
		return 5
	elif education == 'university.degree':
		return 6
	else:
		return 0


# marital 열을 숫자로 매핑
def map_marital(marital):
	if marital == 'married':
		return 1
	elif marital == 'single':
		return 2
	elif marital == 'divorced':
		return 3
	else:
		return 0

# default 열을 0과 1로 매핑
def map_default(default):
	return 1 if default == 'yes' else 0


# housing 열을 0과 1로 매핑
def map_housing(housing):
	return 1 if housing == 'yes' else 0


# loan 열을 0과 1로 매핑
def map_loan(loan):
	return 1 if loan == 'yes' else 0

# contact 열을 숫자로 매핑
def map_contact(contact):
	if contact == 'cellular':
		return 1
	else:
		return 0

# day_of_week 열을 숫자로 매핑
def map_day_of_week(day):
	if day == 'mon':
		return 0
	elif day == 'tue':
		return 1
	elif day == 'wed':
		return 2
	elif day == 'thu':
		return 3
	elif day == 'fri':
		return 4
	elif day == 'sat':
		return 5
	elif day == 'sun':
		return 6

# poutcome 열을 숫자로 매핑
def map_poutcome(poutcome):
	if poutcome == 'failure':
		return 1
	elif poutcome == 'success':
		return 2
	else:
		return 0

# job 열을 숫자로 매핑
def map_job(job):
	job_mapping = {
		'housemaid': 1, 'services': 2, 'admin.': 3, 'technician': 4,
		'blue-collar': 5, 'retired': 6, 'management': 7, 'unemployed': 8,
		'self-employed': 9, 'entrepreneur': 10, 'student': 11
	}
	return job_mapping.get(job, 0)

def exec_convert(train_df):
	if train_df['y'].dtype == 'O':
		train_df['y'] = train_df['y'].apply(map_y)

	# month 열 타입 변경
	if train_df['month'].dtype == 'O':
		train_df['month'] = train_df['month'].map({
			'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5,
			'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10,
			'nov': 11, 'dec': 12
		})
	# education 열을 숫자로 매핑
	if train_df['education'].dtype == 'O':
		train_df['education'] = train_df['education'].apply(map_education)

	# marital 열을 숫자로 매핑
	if train_df['marital'].dtype == 'O':
		train_df['marital'] = train_df['marital'].apply(map_marital)

	# default 열을 0과 1로 매핑
	if 'default' in train_df.columns and train_df['default'].dtype == 'O':
		train_df['default'] = train_df['default'].apply(map_default)

	if train_df['housing'].dtype == 'O':
		train_df['housing'] = train_df['housing'].apply(map_housing)
	if train_df['loan'].dtype == 'O':
		train_df['loan'] = train_df['loan'].apply(map_loan)

	if train_df['contact'].dtype == 'O':
		train_df['contact'] = train_df['contact'].apply(map_contact)

	if train_df['day_of_week'].dtype == 'O':
		train_df['day_of_week'] = train_df['day_of_week'].apply(map_day_of_week)

	if train_df['poutcome'].dtype == 'O':
		train_df['poutcome'] = train_df['poutcome'].apply(map_poutcome)


	if train_df['job'].dtype == 'O':
		train_df['job'] = train_df['job'].apply(map_job)

exec_convert(train_df)


In [ ]:
# 상관관계 히트맵 그리기
corr_matrix = train_df.select_dtypes(include=['int64', 'float64']).corr()

plt.figure(figsize=(15, 8))
sns.set_theme(font_scale=0.8)
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Heatmap', fontsize=14)
plt.show()

In [ ]:
# 이전 캠페인 후 지난 일수에 따른 가입률 시각화
def plot_days_since_last_campaign(df, is_log=False):
	# days_since_last_campaign을 10일 단위로 그룹화
	new_df = df.copy()  # 원본 데이터프레임을 변경하지 않도록 복사
	new_df['days_bin'] = (new_df['pdays'] // 10) * 10
	# 그룹별 가입여부(y) count 집계
	days_counts = new_df.groupby(['days_bin', 'y']).size().unstack(fill_value=0)
	# 가입률(yes 비율) 계산
	days_counts['subscription_rate'] = days_counts[1] / (days_counts[1] + days_counts[0])

	# 시각화
	fig, ax1 = plt.subplots(figsize=(6, 3))

	# count bar
	days_counts[[0, 1]].plot(
		kind='bar', 
		stacked=False, 
		color=[sns.color_palette('Set2')[1], sns.color_palette('Set2')[0]], 
		ax=ax1
	)
	ax1.set_xlabel('Days Since Last Campaign (grouped by 10 days)')
	ax1.set_ylabel('Count')
	if is_log: ax1.set_yscale('log')  # 로그 스케일로 설정
	ax1.set_title('Subscription Count and Rate by Days Since Last Campaign (10 days)')
	ax1.legend(['No', 'Yes'], title='Subscribed')
	ax1.tick_params(axis='x', rotation=45)

	# 가입률(비율) 선그래프
	ax2 = ax1.twinx()
	ax2.plot(days_counts.index.astype(str), days_counts['subscription_rate'], 
			 color=sns.color_palette('Set2')[2], marker='o', label='Subscription Rate')
	ax2.set_ylabel('Subscription Rate')
	ax2.set_ylim(0, 1)
	ax2.legend(loc='upper right')

	plt.tight_layout()
	plt.show()

plot_days_since_last_campaign(train_df)
plot_days_since_last_campaign(train_df, is_log=True)

### 데이터 2차 확인 결과

- 결과
	- 범주형 열의 데이터에 'unknown' 값이 존재함을 확인
	- 직업별 정기 예금 가입률은 유의미해 보이지 않음을 확인
	- 결혼의 유무도 유의미하게 차이나 보이지는 않음
	- 교육 수준별 정기 예금 가입률도 유의미해 보이지는 않음, 'illiterate'는 극소수만이 존재함을 확인
	- 신용불량여부 열
		- 'unknown' 값의 비율이 높지만, 상식적으로 생각해보았을때, 신용불량자가 정기예금을 가입할 가능성은 낮을 것으로 판단됨
		- 모델링시 열 자체를 제거하는 것이 좋을 것으로 판단됨
	- 주택대출 여부는 균등하게 분포되어 있음
	- 개인대출을 받지않은 상담자가 많지만, 정기 예금 가입률은 유의미한 차이를 보이지 않음
	- 연락 유형은 'cellular'가 많고, 정기 예금 가입률도 높음
	- 월별 연락 수는 1, 2월에는 정보가 없음을 확인, 연락 수가 고루 분포되어 있지 않음
	- poutcome 열을 확인 했을때, 'nonexistent'가 많은 것으로 보아 이번 캠페인이 첫 상담인 경우가 많음을 확인
	- age
		- 20대 중반부터 50대 후반까지가 주를 이루고 있음
		- 정기 예금 가입률은 30대 후반에서 40대 초반이 가장 높음
	- duration, campaign열을 보니, 2~3번 연락 중 2~5분의 통화 사이에 정기 예금 가입률이 가장 높음
	- pdays, previous 열을 보니, pdays가 999인 경우가 많고, previous가 0인 경우가 많음
		- pdays가 999인 경우는 이전 캠페인에서 연락이 없었던 경우로 판단됨
		- previous가 0인 경우는 이번 캠페인이 첫 상담인 경우로 판단됨
	- 경제 지표인 emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed 열을 확인했을때 뚜렷한 인사이트는 발견되지 않음

	- age 열은 정기 예금 상품 특성상 나이가 너무 낮거나 많을수록 예금 상품에 가입할 확률이 낮다고 판단, iqr 방식으로 제거
	- 상담 시간은 2~5분 사이에 정기 예금 가입 의사결정이 내려지는 것으로 보임. 이상치 기준을 러프하게 정의하여 10분 이상인 경우 제거
	- 상담 횟수가 너무 많으면 예금 상품에 가입할 확률이 낮다고 판단, iqr 방식으로 제거
	- pdays, previous 열은 이번 캠페인의 신규 상담인경우가 많아 이상치 제거는 하지 않음


- 인사이트
	- 나이와 교육수준, 직업에 따라 정기 예금 가입률의 차이가 있을 것으로 예상됨
	- 이전 캠페인의 결과와 이전 캠페인 연락 횟수, 이전 캠페인 지난 일수와 캠페인 동안 연락 횟수, 마지막 연락일, 정기 예금 가입여부 등 상관관계를 분석할 필요가 있음
	- 소비자 물가지수, 신뢰지수 등 경제 지표와 정기 예금 가입 여부의 상관관계를 분석할 필요가 있음


## 모델링

### 모델 비교

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, BaggingClassifier, StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

"""
confusion_matrix: 실제값 vs 예측값을 교차표로 보여줌

TP (True Positive) : 가입할 사람을 가입한다고 맞춤
TN (True Negative) : 안 할 사람을 안 한다고 맞춤
FP (False Positive) : 안 할 사람을 가입한다고 잘못 예측
FN (False Negative) : 가입할 사람을 안 한다고 놓침

[[TN, FP],
 [FN, TP]]

---

classification_report: 모델의 정밀도(precision), 
					   재현율(recall), 
                       F1 점수, 
                       정확도(accuracy)를 보여주는 요약 리포트 제공

- 단순히 “맞았는지 틀렸는지”만 보는 게 아니라,
	- 얼마나 정확하게 양성 클래스만을 잘 맞췄는지 (precision)
	- 얼마나 놓치지 않고 다 맞췄는지 (recall)
	- 이 두 지표의 조화 평균인 F1-score까지 보여줘서
- 가입/미가입 비율 차이가 많을수록(불균형 데이터) 매우 유용함

---

roc_auc_score: ROC 곡선 아래 면적을 계산하여 모델이 양성과 음성을 얼마나 잘 구별하는지 평가

- 정확도보다 더 균형 잡힌 성능 지표
- 특히 불균형 데이터에서 모델이 "우연히 맞춘 것"인지, 실제로 좋은 구분 능력이 있는지를 알려줌
- 1에 가까울수록 좋가, 0.5면 그냥 찍는 수준

---

predict_proba: 각 클래스에 속할 확률을 반환하는 메서드

"""


# 데이터 분할
X = train_df.drop(
	columns=[
		'y',
	]
)
y = train_df['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def model_test(model, X, model_name):
	if getattr(model, 'feature_importances_', None) is None:
		result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

		sorted_idx = result.importances_mean.argsort()
		plt.barh(range(len(sorted_idx)), result.importances_mean[sorted_idx])
		plt.yticks(np.arange(len(sorted_idx)), X.columns[sorted_idx])
		plt.xlabel("Permutation Importance")
		plt.title(f"{model_name} Feature Importance (Permutation Based)")
		plt.tight_layout()
		plt.show()
	else:

		importances = model.feature_importances_
		feature_names = X.columns

		indices = np.argsort(importances)[::-1]
		plt.figure(figsize=(8, 4))
		sns.barplot(x=importances[indices], y=feature_names[indices], palette='Set2')
		plt.title(f'{model_name} Feature Importance for Subscription Prediction')
		plt.xlabel('Importance')
		plt.ylabel('Features')
		plt.tight_layout()
		plt.show()

# 모델 학습 및 평가 함수 정의
def evaluate_model(model, model_name):
	dt = model.fit(X_train, y_train)

	model_test(dt, X_train, model_name)

	# 예측 및 평가
	y_pred = model.predict(X_test)
	accuracy = accuracy_score(y_test, y_pred)

	print(f"🎯 {model_name} 평가 결과")
	print(f"{model_name} Accuracy: {accuracy:.4f}")
	print(confusion_matrix(y_test, y_pred))
	print(classification_report(y_test, y_pred))
	print("ROC AUC:", roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1]))

	return dt


#### 결정 트리

In [ ]:
model2 = evaluate_model(
    DecisionTreeClassifier(
        max_depth=5, 
        min_samples_split=10,
		min_samples_leaf=5,
        random_state=42
    ), 
    "Decision Tree"
)

#### 부스팅

##### GradientBoosting

In [ ]:
model3 = evaluate_model(
    GradientBoostingClassifier(
        n_estimators=200, 
        learning_rate=0.1, 
        max_depth=3, 
        random_state=42
    ), 
    "Gradient Boosting"
)

##### XGBoost

In [ ]:
model4 = evaluate_model(
    XGBClassifier(
		n_estimators=200,        # 트리 개수
		max_depth=3,             # 트리 깊이 제한
		learning_rate=0.2,       # 학습률
		use_label_encoder=False, # 경고 제거용
		eval_metric='logloss',   # 기본 분류 문제용 손실함수
		scale_pos_weight=1.3,    # 불균형 데이터에서 양성 클래스 가중치 조정
		random_state=42
	),
    "XGBoost"
)

##### LightGBM

In [ ]:
model5 = evaluate_model(
    LGBMClassifier(
		n_estimators=100,
		learning_rate=0.5,
		max_depth=7,
		random_state=42
	),
	"LightGBM"
)

##### CatBoost

In [ ]:
model6 = evaluate_model(
    CatBoostClassifier(
		iterations=200,
		learning_rate=0.8,
		depth=7,
		verbose=0,  # 로그 숨기기 (필요하면 100 단위로 verbose=100도 가능)
		random_state=42
	),
    "CatBoost"
)

#### Clustering

##### KNN

In [ ]:
evaluate_model(
    KNeighborsClassifier(n_neighbors=5),
    "K-Nearest Neighbors"
)

#### 앙상블

In [ ]:
# 앙상블 모델 정의 및 평가
def evaluate_ensemble_model(clf, model_name):
	# 학습
	clf.fit(X_train, y_train)

	y_pred = clf.predict(X_test)
	accuracy = accuracy_score(y_test, y_pred)

	print(f"🎯 {model_name} 평가 결과")
	print(f"{model_name} Accuracy: {accuracy:.4f}")
	print(confusion_matrix(y_test, y_pred))
	print(classification_report(y_test, y_pred))
	print("ROC AUC:", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))


##### Voting

In [ ]:
evaluate_ensemble_model(
	VotingClassifier(
		estimators=[
			('m2', model2),
			('m3', model3),
			('m4', model4),
			('m5', model5),
			('m6', model6),
		],
		voting='soft',  # 확률 평균을 사용한 소프트 보팅
		weights=None,  # 각 모델의 가중치 (None이면 동일 가중치)
        verbose=False,
		n_jobs=-1,  # 모든 CPU 코어 사용
	),
	model_name="Voting Ensemble"
)


##### Bagging

In [ ]:
evaluate_ensemble_model(
    BaggingClassifier(
		estimator=DecisionTreeClassifier(
            max_depth=5,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42
        ),
		n_estimators=200,
		max_samples=0.8,  # 전체 데이터의 80%를 샘플링
		max_features=1.0,  # 모든 특성을 사용
		n_jobs=-1
	),
	model_name="Bagging Ensemble"
)

##### Stacking

In [ ]:
evaluate_ensemble_model(
    StackingClassifier(
		estimators=[
			('m2', model2),
			('m3', model3),
			('m4', model4),
			('m5', model5),
			('m6', model6),
		],
		final_estimator=LogisticRegression(random_state=42), # 최종 추정기는 로지스틱 회귀
		cv=5,  # 5겹 교차 검증
		verbose=0,  # 로그 출력 안함
		n_jobs=-1,  # 모든 CPU 코어 사용
	),
	model_name="Stacking Ensemble"
)

### 모델 평가

|모델|Accuracy|Recall (가입자)|ROC AUC||
|---|---|---|---|--|
|XGBoost|0.9397|0.58|0.9582|종합 최고 성능 ✅|
|Gradient Boosting|0.9380|0.52|0.9554|안정적|
|Decision Tree|0.9344|0.47|0.9263|단순 구조, 성능 낮음|
|CatBoost|0.9301|0.52|0.9343|실용적, 하지만 미세하게 낮음|
|LightGBM|0.9280|0.48|0.9402|recall 낮음|
|Stacking|0.9395|0.52|0.9583|성능 우수하지만 복잡|
|Voting|0.9373|0.51|0.9564||
|Bagging|0.9367|0.48|0.9478||
|KNN|0.9293|0.43|0.8535|성능 가장 낮음|


다양한 분류 모델을 비교한 결과, XGBoost 모델이 가장 우수한 종합 성능을 보였습니다.



### 모델 최적화

In [ ]:
# read dataset
train_df = pd.read_csv("./bank-additional-full.csv", sep=';')

# 중복값 제거 -- 중복값을 제거 한것과 하지 않은것의 차이가 XGBoost 모델의 성능에 큰 영향을 미치는 것으로 확인되었습니다.
train_df.drop_duplicates(inplace=True)

# age 열 iqr 계산 및 제거 -- 정기 예금 상품 특성상 나이가 너무 낮거나 많을수록 예금 상품에 가입할 확률이 낮다고 판단
train_df = remove_age_outliers(train_df, 'age')
# campaign 열 iqr 계산 및 제거 -- 연락을 많이할수록 예금 상품 가입률이 떨어진다고 판단
# train_df = remove_age_outliers(train_df, 'campaign')

# 'unknown' 값을 가진 행 제거
# default 열의 경우 'unknown' 값을 가진 행이 많지만 상식적으로 신용불량인 사람이 정기 예금을 가입하지 않을것으로 판단해 열 자체를 제거
if 'default' in train_df.columns:
	# 'default' 열 제거
	train_df.drop('default', axis=1, inplace=True)

def remove_unknown_rows(df, cat_cols):
	for col in cat_cols:
		df.drop(df[df[col] == 'unknown'].index, inplace=True)

remove_unknown_rows(train_df, cat_cols.drop('default', errors='ignore'))

# education 열에서 'illiterate' 값을 가진 행의 수가 아주 작으므로 제거
train_df.drop(train_df[train_df['education'] == 'illiterate'].index, inplace=True)

# duration > 1300인 행을 이상치로 판단해 제거
train_df.drop(train_df[train_df['duration'] > 1300].index, inplace=True)

# 범주형 열을 숫자로 매핑
exec_convert(train_df)

In [ ]:
# 데이터 분할
X = train_df.drop(
	columns=[
		'y',
		'loan',
		'marital',
		'housing',
        'pdays'
	]
)
y = train_df['y']


# X 에 파생변수 추가
X["total_contacts"] = X["campaign"] + X["previous"] # 총 연락 횟수

X['is_new_customer'] = X['previous'].apply(lambda x: 1 if x == 0 else 0) # 이전 캠페인에 연락한 적이 없는 고객 여부

# 연락 가중치
X["contact_weight"] = (
	- 0.02 * (X['duration'] - 600) ** 2
	- 0.5 * (X['campaign'] - 2) ** 2
	+ 0.1 * (X['previous'] - 1) ** 2
	- 0.2 * (X["campaign"] + X["previous"] - 5) ** 2
)

# 경제지표 가중치
X["cons_weight"] = (
	- 0.02 * (X['nr.employed'] - 5000) ** 2
	- 0.5 * (X['cons.price.idx'] - 2) ** 2
	- 0.5 * (X['cons.conf.idx'] - 2) ** 2
	- 0.5 * (X['euribor3m'] - 2) ** 2
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
evaluate_model(
    XGBClassifier(
		n_estimators=200,        # 트리 개수
		max_depth=3,             # 트리 깊이 제한
		learning_rate=0.2,       # 학습률
		use_label_encoder=False, # 경고 제거용
		eval_metric='logloss',   # 기본 분류 문제용 손실함수
		scale_pos_weight=1.3,    # 불균형 데이터에서 양성 클래스 가중치 조정
		random_state=42
	),
    "XGBoost"
)

1. 데이터 클렌징 및 이상치 제거
	- duration > 1300, age, default, unknown, illiterate 등 제거 → 노이즈 감소 및 일반화 성능 향상
	- 중복 제거가 성능에 실질적 영향을 주었음 → 정확도 개선에 기여
2. 불필요하거나 예측에 방해가 되는 변수 제거
	- loan, housing, marital, pdays 등 제거 -> 모델이 과도하게 의존할 수 있는 “비직접적 변수”들을 제거하여 해석력과 일반화 개선
3. 파생 변수 및 가중치 도입
	- total_contacts, is_new_customer, contact_weight, cons_weight 도입
	- 모델의 미세한 예측 성능 조정 및 AUC 상승에 기여
4. 모델 하이퍼파라미터 조정
	- n_estimators=200, learning_rate=0.2, scale_pos_weight=1.3 등 -> class imbalance 대응 및 학습 최적화


### 결론

여러 차례의 EDA, 파생 변수 생성, 이상치 처리, 변수 선택 및 모델 튜닝을 거친 결과,
최종 XGBoost 모델은 전체 정확도 92.7%, ROC AUC 0.9538로 정확도와 민감도 모두에서 최적의 밸런스를 달성하였습니다.

이 모델은 고객의 정기 예금 가입 여부를 효과적으로 예측할 수 있으며, 마케팅 전략 수립에 유용한 인사이트를 제공할 수 있습니다.

### 제언

- 나이, 직업, 교육 수준, 마지막 연락 지속 시간 등의 특성을 기반으로 맞춤형 마케팅 전략을 수립합니다.
- 1~3회의 연락 횟수와 2~5분 사이 통화시간에 최적화된 승부전략을 수립해야 합니다.
- 'cellular' 연락 유형을 활용하여 고객과의 접점을 늘리고, 정기 예금 상품에 대한 정보를 효과적으로 전달합니다.

- emp.var.rate, cons.price.idx, cons.conf.idx 등의 경제 지표를 모니터링하여 시장 상황에 맞는 마케팅 전략을 수립합니다.
	- 경제 지표가 긍정적인 경우, 정기 예금 상품의 이자율을 조정하여 고객 유치에 유리한 조건을 제공합니다.
	- 부정적인 경우, 고객의 신뢰를 얻기 위해 안정적인 투자 상품으로서 정기 예금의 장점을 강조합니다.

- 모델의 성능을 지속적으로 모니터링하고, 새로운 데이터를 통해 모델을 업데이트합니다.
- 고객의 행동 변화에 따라 모델을 재학습시켜 예측 정확도를 유지합니다.